# Projet IA

In [ ]:
import scipy
import numpy
import sklearn
import pandas as pds
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import sergio_peignier as sp

## Attribute selection

In [ ]:
sparse_df = pds.read_csv("./datas/Data-IA-World-Development-Indicator.txt", sep="\t", header=0)

In [ ]:
sparse_df.shape

In [ ]:
sparse_df.head()

In [ ]:
sparse_df.tail()

In [ ]:
# Un peu de statistiques :
sparse_df.describe()

In [ ]:
#Fonction qui supprime une colonne (attribut) si le nb de NaN >= limit

def delete_NaN_col(df, limit):
    NaN_col = df.isna().sum()
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[1]):
        if NaN_col[i] >= limit:
            df = df.drop(columns = tmp.columns[i])
            
    return(df)

In [ ]:
sparse_df2 = delete_NaN_col(sparse_df, 40)

In [ ]:
# On enlèvre les colonnes 'Time' et 'Time Code' qui ne nous interessent pas ici
sparse_df2 = sparse_df2.drop(columns = ['Time', 'Time Code', 'Country Code'])

In [ ]:
sparse_df2

In [ ]:
sparse_df2.isna().sum()

In [ ]:
sparse_df2.isna().sum(axis=1)

In [ ]:
#Fonction qui supprime une ligne (object) si le nb de NaN >= limit

def delete_NaN_row(df, limit):
    NaN_row = df.isna().sum(axis=1)
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[0]):
        if NaN_row[i] >= limit:
            df = df.drop([i], axis = 0)
            
    return(df)

In [ ]:
sparse_df3 = delete_NaN_row(sparse_df2, 1)

In [ ]:
sparse_df3

In [ ]:
# Nous allons maintenant stocker les noms de pays dans la variable country_labels
# En effet, nous pourrons supprimer la colonne "Country Name" et n'avoir plus que des valeurs numériques.
# Les noms des pays seront toujours stockés dans cette variable et nous pourrons y faire référence dès que nécessaire

country_labels = sparse_df3.loc[:]["Country Name"]

In [ ]:
# On enlèvre la colonnes 'Country Name' 
sparse_df3 = sparse_df3.drop(columns = ['Country Name'])

In [ ]:
corr_df = sparse_df3.corr()
corr_df[:5]

In [ ]:
# Représentation graphique des correlations
plt.figure(figsize= (15, 12))
sns.heatmap(corr_df,annot=True)
plt.show()

In [ ]:
sns.clustermap(corr_df,
               figsize= (16, 12),
               annot=True,
               dendrogram_ratio=(0.1, 0.2),
               row_cluster=False)
plt.show()

In [ ]:
def delete_corr(df, limit):
    #On construit notre matrice de correlation en valeur absolue
    corr_matrix = df.corr().abs()
    
    #Triangle supérieur de la matrice de corrélation : 
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > limit)]
    df = df.drop(df[to_drop], axis=1)
    return(df)

In [ ]:
df = delete_corr(sparse_df3, 0.55)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize= (15, 12))
sns.heatmap(df.corr().abs(), annot = True)
plt.show()